# Chat Memory

### Import Library

In [1]:
from haystack_experimental.chat_message_stores.in_memory import InMemoryChatMessageStore
from haystack_experimental.components.retrievers import ChatMessageRetriever
from haystack_experimental.components.writers import ChatMessageWriter
from haystack.dataclasses import ChatMessage
from haystack.components.joiners import ListJoiner
from haystack import Pipeline
from typing import List
from haystack.components.builders import ChatPromptBuilder, PromptBuilder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.generators import OpenAIGenerator
from haystack.components.converters import OutputAdapter
from haystack.utils import Secret
from getpass import getpass
import os

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["MONGO_CONNECTION_STRING"]
os.environ['OPENAI_API_KEY']

### Membuat Pipeline

membuat chat tanpa memory

In [ ]:
system_message = ChatMessage.from_system(
    "You are a helpful assistant that answers questions based on the provided context."
)
user_message_template = """
Answer the question based on the user query:
query:{{query}}
answer:
"""
user_message = ChatMessage.from_user(user_message_template)

In [4]:
pipeline = Pipeline()
pipeline.add_component("prompt_builder", ChatPromptBuilder(
    variables=["query"], 
    required_variables=["query"]
))
pipeline.add_component("generator", OpenAIChatGenerator(
    model="gpt-4.1", 
    api_key=Secret.from_token(os.environ["OPENAI_API_KEY"])
))
pipeline.connect("prompt_builder.prompt", "generator.messages")

🚅 Components
  - prompt_builder: ChatPromptBuilder
  - generator: OpenAIChatGenerator
🛤️ Connections
  - prompt_builder.prompt -> generator.messages (List[ChatMessage])

In [5]:
while True:
    messages = [system_message, user_message]
    query = input("Please input your question or type 'exit' to quit.\n")
    if query.lower() == "exit":
        break
    res = pipeline.run(
        {
            "prompt_builder": {
                "query": query,
                "template":messages
            }
        }
    )
    print("AI Response:", res["generator"]["replies"][0].text)

AI Response: Albert Einstein was a German-born theoretical physicist who is best known for developing the theory of relativity, one of the two pillars of modern physics (the other being quantum mechanics). His most famous equation is E = mc^2, which expresses the relationship between mass and energy. Einstein was awarded the Nobel Prize in Physics in 1921 for his explanation of the photoelectric effect. He made many other contributions to science, and his work has had a profound impact on the way we understand the universe.
AI Response: I'm sorry, but I need more information about who "he" refers to in your question. Could you please provide the name or more context?


Membuat Pipeline dengan memory chat

mendefinisikan InMemoryChatMessageStore terlebih dahulu

In [10]:
memory_store = InMemoryChatMessageStore()
memory_retriever = ChatMessageRetriever(memory_store)
memory_writer = ChatMessageWriter(memory_store)

In [7]:
system_message = ChatMessage.from_system("You are a helpful assistant that answers questions based on the provided context.")
user_message_template = """
Answer the question based on the user query, please pay attention to the chat history:
chat_history:
{% for memory in memories %}
    {{memory.text}}
{% endfor %}

query:{{query}}
answer:
"""
user_message = ChatMessage.from_user(user_message_template)

In [11]:
pipeline = Pipeline()
pipeline.add_component("prompt_builder", ChatPromptBuilder(
    variables=["query","memories"], 
    required_variables=["query","memories"]
))
pipeline.add_component("generator", OpenAIChatGenerator(
    model="gpt-4.1", 
    api_key=Secret.from_token(os.environ["OPENAI_API_KEY"])
))
pipeline.add_component("joiner", ListJoiner(List[ChatMessage]))
pipeline.add_component("memory_retriever", memory_retriever)
pipeline.add_component("memory_writer", memory_writer)

pipeline.connect("prompt_builder.prompt", "generator.messages")
pipeline.connect("generator.replies", "joiner")
pipeline.connect("joiner", "memory_writer")
pipeline.connect("memory_retriever", "prompt_builder.memories")

🚅 Components
  - prompt_builder: ChatPromptBuilder
  - generator: OpenAIChatGenerator
  - joiner: ListJoiner
  - memory_retriever: ChatMessageRetriever
  - memory_writer: ChatMessageWriter
🛤️ Connections
  - prompt_builder.prompt -> generator.messages (List[ChatMessage])
  - generator.replies -> joiner.values (List[ChatMessage])
  - joiner.values -> memory_writer.messages (List[ChatMessage])
  - memory_retriever.messages -> prompt_builder.memories (List[ChatMessage])

In [ ]:
while True:
    messages = [system_message, user_message]
    query = input("Please input your question or type 'exit' to quit.\n")
    if query.lower() == "exit":
        break
    res = pipeline.run(
        data={
            "prompt_builder": {
                "query": query,
                "template":messages
            },
            "joiner":{
                "values": [ChatMessage.from_user(query)]
            }
        },
        include_outputs_from=["generator"]
    )
    print("AI Response:", res["generator"]["replies"][0].text)

AI Response: Albert Einstein was a renowned German-born theoretical physicist best known for developing the theory of relativity, one of the two pillars of modern physics (the other being quantum mechanics). His famous equation, E=mc², expresses the relationship between mass and energy. Einstein received the Nobel Prize in Physics in 1921 for his explanation of the photoelectric effect, which was pivotal in establishing quantum theory. He is widely regarded as one of the most influential scientists of the 20th century.
AI Response: Albert Einstein was born on March 14, 1879.
AI Response: Goodbye! If you have any more questions in the future, feel free to ask.
